# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [10]:
#python array handling
import xarray
import numpy
import cftime
import pandas

In [1]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [5]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [6]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in _index.monthsOfInterest:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino34NoDetrend,nino4NoDetrend,eastIONoDetrend,dmi,nino34,nino4,eastIO,sam,lat1,lat2,precAnom,tsAnom,pr,tas,enso,eIndex,cIndex,backgroundSstNoDetrend
year,,,,,,,,,,,,,,,,,,
1995,1.67126,1.63732,2.85324,3.10786,2.55862,2.75747,3.23472,1.68724,2.38563,1.89391,1.66268,2.63662,3.01006,3.03770,2.94753,2.17710,3.31113,2.86480
1996,2.17010,3.06303,1.69822,3.05494,2.40731,2.24308,2.79486,1.71159,2.71981,3.00651,2.34615,2.44731,2.38070,2.70765,3.46313,3.52662,2.19462,2.69605
1997,3.07137,2.63338,3.79813,2.80878,3.24515,2.07645,2.61894,2.26748,1.88447,1.70176,2.96224,2.69245,3.35593,2.50843,3.29195,2.21439,3.20344,2.63381
1998,2.82024,2.83808,2.47158,2.07007,3.59592,2.90395,2.78870,1.25994,2.69727,2.45403,2.39106,1.84813,3.16388,1.84690,2.40939,2.81989,3.46811,1.86276


Calculate it using the function

In [7]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)

calculatedAvs.round(5)

<xarray.Dataset>
Dimensions:                 (year: 4)
Coordinates:
  * year                    (year) int64 1995 1996 1997 1998
Data variables: (12/18)
    cIndex                  (year) float64 3.311 2.195 3.203 3.468
    eIndex                  (year) float64 2.177 3.527 2.214 2.82
    lat1                    (year) float64 2.386 2.72 1.884 2.697
    nino4NoDetrend          (year) float64 1.637 3.063 2.633 2.838
    pr                      (year) float64 3.01 2.381 3.356 3.164
    sam                     (year) float64 1.687 1.712 2.267 1.26
    ...                      ...
    eastIONoDetrend         (year) float64 2.853 1.698 3.798 2.472
    backgroundSstNoDetrend  (year) float64 2.865 2.696 2.634 1.863
    eastIO                  (year) float64 3.235 2.795 2.619 2.789
    precAnom                (year) float64 1.663 2.346 2.962 2.391
    enso                    (year) float64 2.948 3.463 3.292 2.409
    nino34NoDetrend         (year) float64 1.671 2.17 3.071 2.82

In [8]:
numpy.isclose(calculatedAvs.sam, expectedAv.sam)

array([ True,  True,  True,  True])

In [9]:
numpy.unique(testXr.time.dt.year)

array([1995, 1996, 1997, 1998, 1999])

# Example Seasonal Averaging - CESM

An example of using it with CESM Data

Load some example data

In [ ]:
pslDs = fh.loadModelData('CESM-LME', 'PSL', '002')
sstDs = fh.loadModelData('CESM-LME', 'SST', '002')

Calculate some indices

In [ ]:
pslIndex = psl.calculateSamIndex(pslDs, 850, 2005)
sstIndex = sst.calculateIndex(sstDs)

In [ ]:
pslIndex

In [ ]:
sstIndex

In [ ]:
indeces = xarray.merge([pslIndex, sstIndex])
indeces

In [ ]:
indeces.to_netcdf('results/exampleMonthlyIndeces.nc')

Run the averaging

In [ ]:
indeces = xarray.load_dataset('results/exampleMonthlyIndeces.nc')

In [ ]:
results = averageForTimePeriod(indeces)


In [ ]:
results